# start to play !

In [34]:
from mxnet.gluon.data.vision import ImageFolderDataset
import cv2
import numpy as np
from mxnet import ndarray as nd
class pyyData(ImageFolderDataset):
    def __init__(self, *arg1, **arg2):
        super(pyyData,self).__init__(*arg1,**arg2)

    def __getitem__(self, idx):
        '''
        use cv2 backend
        '''
        img  = cv2.imread(self.items[idx][0])
        if img.shape[2] == 1:
            # read an gray map
            img = cv2.cvtcolor(img,cv2.color_gray2rgb)
        img  = nd.array( img[:,:,:3]).astype(np.uint8)
        label = self.items[idx][1]
        if self._transform is not None:
            return self._transform(img, label)
        return img, label
# pyy = pyyData(test_path)


transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor()
#     ,transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_path = "./classified-images"
batch_size = 512
num_gpus = 1
num_workers = 8
ctx = [mx.gpu(i) for i in range(num_gpus)] if num_gpus > 0 else [mx.cpu()]

test_data = gluon.data.DataLoader(
    pyyData(test_path).transform_first(transform_test),
    batch_size=batch_size, shuffle=True, num_workers = num_workers,last_batch = 'discard')

# 预训练模型下载

In [35]:
from mxnet.gluon import nn
from mxnet import gluon, image, init, nd
model_name = 'ResNet50_v1d'
finetune_net = gluoncv.model_zoo.get_model(model_name, pretrained=True)

with finetune_net.name_scope():
    finetune_net.fc = nn.Dense(2)
finetune_net.fc.initialize(init.Xavier(), ctx = ctx)
finetune_net.collect_params().reset_ctx(ctx)
finetune_net.hybridize()

In [1]:
import time
t0 = time.time()
ctx  = [mx.gpu(0)]
for idx , g in enumerate(test_data):
    g[0] = g[0].as_in_context(mx.gpu(0))
    y = finetune_net(g[0])
    print(y.shape)